In [60]:
from pydantic import BaseModel, Field
from typing import List
from enum import Enum


class Descripcion(BaseModel):
    descripcion: str = Field(
        ..., description="Descripción proporcionada por el usuario"
    )


class CategoriaEnum(str, Enum):
    peticiones = "peticiones"
    quejas = "quejas"
    reclamos = "reclamos"


class Categoria(BaseModel):
    categoria: CategoriaEnum = Field(..., description="Categoría principal clasificada")


class Subcategoria(BaseModel):
    subcategoria: str = Field(
        ..., description="Subcategoría seleccionada dentro de la categoría principal"
    )


class Resultado(BaseModel):
    categoria: Categoria
    subcategoria: Subcategoria


In [61]:
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
import load_dotenv
import openai
import os
from langchain_openai import ChatOpenAI
import pandas as pd


load_dotenv.load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")
# Configurar el modelo OpenAI con tu API key
llm = ChatOpenAI(model="gpt-3.5-turbo-0125")

# Definir las categorías y subcategorías
subcategorias_peticiones = [
    "Consulta de información",
    "Aclaración de información y movimiento",
    "Certificaciones/paz y salvos",
    "Copia de pólizas",
    "Estados de cuenta, extracto y reportes",
    "Información errada producto-servicio",
    "Protección de datos",
    "Liquidación de intereses",
    "Acumulación/redención davipuntos-millas",
    "Felicitación a funcionario",
]

subcategorias_quejas = [
    "Inconformidad en débito y gestión",
    "Inconformidad en el saldo y gestión cobranza",
    "Inconformidad Tasa Hipoteca/Leasing y venta",
    "Notificaciones y reclamaciones erradas",
    "Indemnización seguro",
    "Inconsistencias de pagos y compras",
    "Cobro errado cuotas seguros",
    "Mal funcionamiento de canales",
    "Inconformidad en la atención",
    "Inconformidad tarifas y comisiones",
]

subcategorias_reclamos = [
    "Inconsistencias de transacciones generales",
    "Ajuste tasa hipoteca y asegurado",
    "Descuento no pago, descontento y no pago atm propio",
    "Pagos y recaudos mal aplicados",
    "Diferencia en justes saldos/intereses",
    "Embargos y deseembargos",
    "Levantamiento hipoteca",
    "Problemas con la Tarjeta de crédito",
    "Venta no abonada",
]

# Crear un diccionario para las subcategorías
subcategorias_dict = {
    "peticiones": subcategorias_peticiones,
    "quejas": subcategorias_quejas,
    "reclamos": subcategorias_reclamos,
}


In [62]:
clasificacion_prompt = PromptTemplate(
    input_variables=["descripcion"],
    template="Clasifica la siguiente descripción en una de las siguientes categorías: 'peticiones', 'quejas', 'reclamos', 'categoria no identificada'. Solo puedes usar una de esas opciones, no agregues ninguna otra palabra.\n\nDescripción: {descripcion}",
)

clasificacion_chain = LLMChain(llm=llm, prompt=clasificacion_prompt)

subcategoria_prompt = PromptTemplate(
    input_variables=["descripcion", "categoria"],
    template="""
    La descripción pertenece a la categoría {categoria}. Ahora, selecciona la subcategoría más adecuada de la siguiente lista:
    {subcategorias}

    Descripción: {descripcion}
    """,
)


In [63]:

def seleccionar_subcategoria(descripcion, categoria):
    subcategorias = "\n".join(subcategorias_dict[categoria])
    return subcategoria_prompt.format(
        descripcion=descripcion,
        categoria=categoria,
        subcategorias=subcategorias
    )

subcategoria_chain = LLMChain(llm=llm, prompt=subcategoria_prompt)

def procesar_descripcion(descripcion_input: Descripcion) -> Resultado:
    # Validar y extraer la descripción
    descripcion = descripcion_input.descripcion
    
    # Clasificar la descripción en una categoría principal
    categoria_response = clasificacion_chain.invoke({"descripcion": descripcion})

    categoria_str = categoria_response['text'].strip().replace(".", "").lower()
    categoria = Categoria(categoria=CategoriaEnum(categoria_str))
    
    # Seleccionar la subcategoría dentro de la categoría principal
    subcategoria_prompt_str = seleccionar_subcategoria(descripcion, categoria.categoria.value)
    subcategoria_response = subcategoria_chain.invoke({"descripcion": descripcion, "categoria": categoria.categoria.value, "subcategorias": subcategoria_prompt_str})

    subcategoria_str = subcategoria_response['text'].strip()
    subcategoria = Subcategoria(subcategoria=subcategoria_str)
    # # Crear el resultado final
    resultado = Resultado(categoria=categoria, subcategoria=subcategoria)
    return resultado

# Ejemplo de uso
# descripcion_input = Descripcion(descripcion="Necesito saber el saldo de mi cuenta y obtener un extracto mensual.")
# resultado = procesar_descripcion(descripcion_input)
# print(f"Categoría: {resultado.categoria.categoria.value}, Subcategoría: {resultado.subcategoria.subcategoria}")

In [64]:
DATA = {
    "empresas": "Empresas 2da Línea",
    "no_fraude": "No Fraude",
}

base = "empresas"
df = pd.read_excel("../../data/raw/Cifras Analitica.xlsx", sheet_name="Cifras")
df = df.query(
    f"MESA == '{DATA[base]}' & PRODUCTO != '' & PRODUCTO != 0 & CANAL_ORIGEN == 'Clientes'"
)


In [65]:
import logging
logging.basicConfig(level=logging.ERROR)


categories = []
subcategories = []
for index, row in df.iterrows():
    print(f"Processing row {index}")
    descripcion_input = Descripcion(descripcion=row["DESCRIPCION"])
    resultado = procesar_descripcion(descripcion_input)
    categories.append(resultado.categoria.categoria.value)
    subcategories.append(resultado.subcategoria.subcategoria)


Processing row 9
Processing row 28
Processing row 39
Processing row 58
Processing row 83
Processing row 113
Processing row 140
Processing row 255
Processing row 270
Processing row 334
Processing row 340
Processing row 342
Processing row 355
Processing row 357
Processing row 368
Processing row 378
Processing row 379
Processing row 400
Processing row 416


KeyboardInterrupt: 

In [4]:
import pandas as pd
df = pd.DataFrame()
df["category"] =  ['cat','dog','cat']
filtered_df = df[df["category"] == 'cagt']
if df[df["category"] == 'cagt'].empty:
    print('empty')

empty


In [ ]:
df.shape

(346, 32)